In [1]:
library(tidyverse)
library(dplyr)
library(repr)
library(tidymodels)
library(recipes)
install.packages("themis")
library(themis)

Warning message:
“package ‘ggplot2’ was built under R version 4.3.2”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_co

## Introduction

- Wine quality highly depends on its composition of various aromatic compounds. By assessing the content of wines such as their sugar content and pH value, we will be able to determine how good the wine is. 
- In this project, we will be using the wine quality dataset obtained from a web URL.
- We want to solve **regression problem** —- what would be the wine’s quality level given its values on the fixed acidity, volatile acidity, and other variables.
-  7 out of the 12 variables are used to predict the **wine quality**, which ranges from 0 to 10, with 10 being the highest quality wine.
- We focus on indicating which attributes affect the wine quality and predicting the quality based on each attribute's values. By using the regression method, we create a linear formula and use the coefficients to know how the attribute level influences the wine quality(i.e. it positively/negatively influences the wine quality).

## Preliminary exploratory data analysis

**1. Data Reading**

In [2]:
temp <- tempfile()
download_file <- download.file("https://archive.ics.uci.edu/static/public/186/wine+quality.zip", temp)
red_wine <- read.csv2(unz(temp, "winequality-red.csv")) |>
             mutate(type = 0)
white_wine <- read.csv2(unz(temp, "winequality-white.csv")) |>
              mutate(type = 1)
unlink(temp)

**2. Data Wrangling**

In [3]:
wine_data <- rbind(red_wine, white_wine) |>
             mutate(across(fixed.acidity:alcohol, as.numeric), quality = ifelse(quality <= 5, 0, 1),
                   type = as_factor(type))

write_csv(wine_data, "../data/wine_data.csv")

**3. Data Splitting**

To know how good our prediction is, we should split our data into training dataset and testing dataset.

In [4]:
set.seed(2000)
wine_split <- initial_split(wine_data, prop = 0.75, strata = quality)
wine_training <- training(wine_split)
wine_testing <- testing(wine_split)

**4. Summarization**

**a. The number of observations in each red wine/white wine class**

In [5]:
wine_type_number <- wine_training |>
                    group_by(type) |>
                    summarize(Number = n())
wine_type_number

- We have 4898 observations for White wine and 1599 observations for Red wine.
- There is an imbalance between the types of the wine. 

**b. The number of observations for each quality level.**

In [6]:
quality_number <- wine_training |>
                  group_by(quality) |>
                  summarize(Number = n())
quality_number

**c. Mean value of each measurements for different quality type.**

In [7]:
mean_measurements <- wine_training |>
                     group_by(quality) |>
                     summarize(mean_fixed_acidity = mean(fixed.acidity),
                               mean_volatile_acidity = mean(volatile.acidity),
                               mean_citric_acid = mean(citric.acid),
                               mean_residual_sugar = mean(residual.sugar),
                               mean_chlorides = mean(chlorides),
                               mean_free_sufdioxide = mean(free.sulfur.dioxide),
                               mean_tot_sufdioxide = mean(total.sulfur.dioxide),
                               mean_density = mean(density),
                               mean_ph = mean(pH),
                               mean_sulphates = mean(sulphates),
                               mean_alcohol = mean(alcohol))
mean_measurements

The ranges of the mean are different across different columns hence the variables will need to be scaled 

**d. Missing data - We have no missing data in this dataset.**

**4. Visualization: histogram between quality and other numeric variables**

In [8]:
wine_visualization <- wine_training |>
                      mutate(quality = as.factor(quality))  
histograms <- lapply(names(wine_visualization)[!names(wine_visualization) %in% c("quality", "type")], function(col) {
  ggplot(wine_visualization, aes(x = !!sym(col), fill = quality)) +
    geom_histogram(position = "dodge", bins = 100, binwidth = 2) +
    labs(title = paste("Histogram of", col), x = col, y = "Counts", fill = "Quality") +
    theme_minimal()
})

histograms

- Most variables have a linear negative correlation to quality
- free.sulfur.dioxide and total.sulfur. dioxide seems to have no correlation with quality as the categories' bins are layered in the histogram.
- alcohol and residual.sugar have a non-linear relationship with quality

## Training & Evaluation

### 1. The multivariable Linear regression Model

In [9]:
set.seed(1)
wine_data_reg <- wine_training |>
                  mutate(quality = as.numeric(quality)) |>
                  select(alcohol, chlorides, citric.acid, sulphates, type, quality)
head(wine_data_reg)

In [10]:
lm_recipe <- recipe(quality ~ .,
                     data = wine_data_reg)

lm_spec <- linear_reg() |>
   set_engine("lm") |>
   set_mode("regression")

lm_fit <- workflow() |>
   add_recipe(lm_recipe) |>
   add_model(lm_spec) |>
   fit(data = wine_data_reg)

 lm_fit

In [11]:
lm_test_results <- lm_fit |>
   predict(wine_testing) |>
   bind_cols(wine_testing) |>
   metrics(truth = quality, estimate = .pred)

 lm_test_results

 lm_coeffs <- lm_fit |>
              extract_fit_parsnip() |>
              tidy()
 lm_coeffs

The RMSPE of this linear regression model is 0.7961641

The mathematical expression to describe this linear regression model is:
$quality = 3.81887647 + 0.09180436 \cdot (fixed.acidity) - 0.19801129 \cdot (volatile.acidity) + 0.01694675 \cdot (citric.acid) - 0.08928023 \cdot (chlorides) - 0.08928023 \cdot (density) + 0.07583322 \cdot (pH) + 0.13697853 \cdot (sulphates)$

### Visualization of the result

### 2. K-nn regression Model

In [12]:
# knn_recipe <- recipe(quality ~ ., data = wine_data_reg) |>
#   step_scale(all_numeric_predictors()) |>
#   step_center(all_numeric_predictors())

In [13]:
# wine_knnreg_spec <- nearest_neighbor(weight_func = "rectangular",
#                                      neighbors = tune()) |>
#                     set_engine("kknn") |>
#                     set_mode("regression")

In [14]:
# knn_vfold <- vfold_cv(wine_training, v = 5, strata = quality)

# knn_workflow <- workflow() |>
#     add_recipe(knn_recipe) |>
#     add_model(wine_knnreg_spec)

# knn_workflow

In [15]:
# gridvals <- tibble(neighbors = seq(from = 10, to = 100, by = 10))

# knn_reg_results <- knn_workflow |>
#   tune_grid(resamples = knn_vfold, grid = gridvals)
#show the results
#head(knn_reg_results)

In [16]:
# knn_min <- knn_reg_results |>
#     filter(mean == min(mean))
# knn_min

The smallest RMSPE occurs when K = 19.

In [17]:
# kmin <- knn_min |> pull(neighbors)

# knn_reg_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = kmin) |>
#   set_engine("kknn") |>
#   set_mode("regression")

# knn_reg_fit <- workflow() |>
#   add_recipe(knn_recipe) |>
#   add_model(knn_reg_spec) |>
#   fit(data = wine_reg_training)

# knn_reg_summary <- knn_reg_fit |>
#   predict(wine_reg_testing) |>
#   bind_cols(wine_reg_testing) |>
#   metrics(truth = quality, estimate = .pred) |>
#   filter(.metric == 'rmse')

# knn_reg_summary

### 2. Classification Model

In [18]:
wine_data_class <- wine_data |>
    select(alcohol, chlorides, citric.acid, sulphates, type, quality)

wine_data_class$quality <- as.factor(wine_data_class$quality)

Before doing the classification model, we need to centering, scaling, and balance the data first.

In [19]:
# Split the data
wine_class_split <- initial_split(wine_data_class, prop = 0.6, strata = quality)
wine_class_training <- training(wine_class_split)
wine_class_testing <- testing(wine_class_split)

In [20]:
# Create a recipe
knn_recipe <- recipe(quality ~ ., data = wine_class_training) |>
    step_scale(all_numeric_predictors()) |>
    step_center(all_numeric_predictors()) |>
    step_upsample(quality, over_ratio = 1, skip = TRUE) 

In [21]:
prepped_train_wine <- knn_recipe |> 
                      prep(data = wine_class_training) |> 
                      bake(new_data = wine_class_training)

ERROR: [1m[33mError[39m in `step_scale()`:[22m
[1mCaused by error in `prep()`:[22m
[33m![39m Can't subset columns that don't exist.
[31m✖[39m Column `all_numeric_predictors` doesn't exist.


In [ ]:
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
            set_engine("kknn") |>
            set_mode("classification")

In [ ]:
knn_recipe_final <- recipe(quality ~ ., data = prepped_train_wine) 

In [ ]:
wine_vfold <- vfold_cv(wine_class_training, v = 5, strata = quality)

In [ ]:
gridvals <- tibble(neighbors = seq(from = 10, to = 50, by = 10))

In [ ]:
knn_fit <- workflow() |>
                add_recipe(knn_recipe_final) |>
                add_model(knn_spec) |>
                tune_grid(resamples = wine_vfold, grid = gridvals)

In [ ]:
best_k <- knn_fit |>
          select_best("accuracy") |>
          pull(neighbors)  

In [ ]:
knn_spec_final <- nearest_neighbor(weight_func = "rectangular", neighbors = 10) |>
             set_engine("kknn") |>
             set_mode("classification")

In [ ]:
knn_fit_final <- workflow() |>
          add_recipe(knn_recipe_final) |>
          add_model(knn_spec_final) |>
          fit(data = prepped_train_wine)

In [ ]:
knn_pred <- predict(knn_fit_final, new_data = wine_class_testing) |>
            bind_cols(wine_class_testing)

In [ ]:
knn_pred_numeric <- knn_pred |>
                      mutate(.pred_class = as.numeric(as.character(.pred_class)), quality = as.numeric(quality) - 1)
knn_pred_numeric
rmse <- knn_pred_numeric |>
        metrics(truth = quality, estimate = .pred_class) 
rmse

The accuracy observed in this classification model is 0.603936, and the RMSE in this classificaiton model is 0.849354

## Methods

- After data reading and wrangling, we used all 12 variables for preliminary data summarization and visualization. We then compared the number of observations in each red/ white wine class, the number of observations in each quality level for red and white wine respectively, and the mean value of each feature for each quality level. 

- Multi-histogram is used to visualize the relationship between each variable and the wine quality, each with one predictor on the x axis, counts on the y axis, and color indicating the different quality levels. If each quality level tends to show up on a certain value of the x axis, we can conclude that there is linear correlation between the wine quality and that predictor. Otherwise, there is no correlation or non-linear correlation. 

- **We will only use 7 numeric variables excluding type total.sulfur.dioxide, free.sulfur.dioxide, residual sugar and alcohol** 

## Expected outcomes and significance

- Due to class imbalance in the categories of the target column (quality): some of the categories are missing. We expect the model will be less precise for the level from 0 to 2 and the level 10.
- However, we hope the model will be able to predict correctly the quality from 3 - 8

- These findings will allow a more efficient and more systematic wine quality assessment as we will be able assess the quality of the wines not only by its age or individual tastes, but also the actual content in the wine itself.
-  The findings can also lead to further discoveries such as how different environments in which the grapes grew in would affect the quality of the wines, in which they can be used to optimize the wine development.
